In [1]:
import pandas as pd

In [2]:
# Labels from https://www.nature.com/articles/s41586-021-03493-4 Stukalov et al. 2021
# Direct download: https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-021-03493-4/MediaObjects/41586_2021_3493_MOESM6_ESM.xlsx
df = pd.read_excel('41586_2021_3493_MOESM6_ESM.xlsx', sheet_name='A - Significant interactions')
df

/Users/yang.an/mamba/envs/tf2/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/yang.an/mamba/envs/tf2/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,bait_organism,bait_name,gene_name,protein_description,majority_protein_acs,fold_change_log2,p_value,fold change_threshold,comparison_batch,fold_change_log2_batch,...,oeproteome_p_value,oeproteome_is_hit,rnaseq_comparison,rnaseq_is_hit,proteome_comparison,proteome_is_hit,phospho_comparison,phospho_is_hit,ubi_comparison,ubi_is_hit
0,SARS-CoV,E,UFD1,Ubiquitin recognition factor in ER-associated ...,Q92890;Q92890-1;Q92890-3,9.969620,6.016913e-276,2,SARS_CoV_E_vs_B5_others,10.077186,...,0.445602,0.0,NaN,0.0,NaN,0.0,S247@UFD1[24h:shared(-) 36h:shared(-)] S299@UF...,1.0,NaN,0.0
1,SARS-CoV,E,NPLOC4,Nuclear protein localization protein 4 homolog...,Q8TAT6;Q8TAT6-2,8.854112,3.193217e-20,2,SARS_CoV_E_vs_B5_others,9.110157,...,0.009037,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0
2,SARS-CoV,E,SARS_CoV_E,NO DESCRIPTION,P59637,9.536133,1.491272e-68,2,SARS_CoV_E_vs_B5_others,9.622996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,K63@SARS_CoV_E[12h:SARS_CoV(+) 24h:SARS_CoV(+)],1.0
3,SARS-CoV,E,RAB34,"Ras-related protein Rab-34, isoform NARR",P0DI83,7.876418,4.422959e-11,2,SARS_CoV_E_vs_B5_others,8.156219,...,0.052086,0.0,NaN,0.0,NaN,0.0,S123@RAB34[36h:shared(-)],1.0,NaN,NaN
4,SARS-CoV,E,FAM241B,Protein FAM241B...,Q96D05;Q96D05-2,6.863366,3.467090e-04,2,SARS_CoV_E_vs_B5_others,6.953465,...,0.096263,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,SARS-CoV,S,ZDHHC5,Palmitoyltransferase ZDHHC5...,Q9C0B5;Q9C0B5-2,7.105674,3.793086e-04,2,SARS_CoV_S_vs_B4_others,7.465508,...,0.760743,0.0,NaN,0.0,NaN,0.0,T348@ZDHHC5[12h:shared(+) 24h:SARS_CoV2(+) 36h...,1.0,NaN,0.0
2102,SARS-CoV,S,SARS_CoV_S,Spike glycoprotein,P59594,16.554913,0.000000e+00,2,SARS_CoV_S_vs_B4_others,16.492960,...,NaN,NaN,NaN,NaN,NaN,0.0,T215@SARS_CoV_S[36h:SARS_CoV(+)],1.0,K188@SARS_CoV_S[12h:SARS_CoV(+) 24h:SARS_CoV(+...,1.0
2103,SARS-CoV-2,S,ZDHHC5,Palmitoyltransferase ZDHHC5...,Q9C0B5;Q9C0B5-2,7.446649,7.353729e-06,2,SARS_CoV2_S_vs_B5_others,7.662380,...,0.565619,0.0,NaN,0.0,NaN,0.0,T348@ZDHHC5[12h:shared(+) 24h:SARS_CoV2(+) 36h...,1.0,NaN,0.0
2104,SARS-CoV-2,S,GOLGA7,Golgin subfamily A member 7...,Q7Z5G4;Q7Z5G4-3,7.566474,1.689030e-04,2,SARS_CoV2_S_vs_B5_others,7.732983,...,0.218128,0.0,NaN,0.0,NaN,0.0,NaN,NaN,K95@GOLGA7[24h:shared(+)],1.0


In [3]:
# Extract SARS-CoV host factors
df_sars = df[~df['gene_name'].str.contains('SARS')]
df_sars = df_sars[df_sars['bait_organism'] == 'SARS-CoV']['majority_protein_acs'].str.split(';').explode()
df_sars[~df_sars.str.contains('-') & ~df_sars.str.contains('SARS')].nunique()

612

In [4]:
predictions = pd.read_csv('../benchmark/prediction_values_all_models.csv.zip')

In [5]:
predictions['SARS-CoV'] = predictions['protein_ac'].isin(df[df['bait_organism'] == 'SARS-CoV']['majority_protein_acs'].str.split(';').explode().tolist())

In [6]:
# ideal threshold
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, accuracy_score, precision_score, average_precision_score, recall_score, precision_recall_curve
import numpy as np

modality = 'tf_finetuned'
config = 'tf_finetuned_ensemble'
Ks = [50, 100, 200]
results = []

cov = 'SARS-CoV'
val_subset = predictions[predictions[f'group_split_0'].isin(['train', 'val'])].copy()

precision, recall, thresholds = precision_recall_curve(val_subset['label'] == '1.0', val_subset[f'{modality}_ensemble'])
f1_scores = 2*precision*recall / (precision+recall)
best_threshold = thresholds[f1_scores.argmax()]

test_subset = predictions.copy()

result = {}
result.update({
          'Virus': cov,
          f'AUROC': roc_auc_score(test_subset[cov], test_subset[config]),
          f'APS': average_precision_score(test_subset[cov], test_subset[config]),
          f'F1-score': f1_score(test_subset[cov], test_subset[config] > best_threshold, ),
          f'Precision': precision_score(test_subset[cov], test_subset[config] > best_threshold, ),
          f'Recall': recall_score(test_subset[cov], test_subset[config] > best_threshold, ),
         })

for K in Ks:
    test_subset_top_k = test_subset.loc[test_subset[config].nlargest(K).index]
    result.update({f'Precision@{K}': precision_score(test_subset_top_k[cov], np.ones_like(test_subset_top_k[config]))})


results.append(result)

results = pd.DataFrame(results)
results.round(2)

,Virus,AUROC,APS,F1-score,Precision,Recall,Precision@50,Precision@100,Precision@200
0,SARS-CoV,0.8,0.11,0.19,0.12,0.42,0.16,0.19,0.2
